In [2]:
import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import math
import h5py

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from shutil import copyfile
from copy import deepcopy

from code_base.utils import write_json

/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2024

In [5]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended.csv", converters={"secondary_labels":eval})

In [6]:
all_birds = list(set(train_metadata["primary_label"]) | set(chain(*train_metadata["secondary_labels"])))

In [7]:
bird2int = {
    bird:idx for idx, bird in enumerate(all_birds)
}

In [8]:
write_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json", bird2int)

# 2024 Scored Data DF

In [3]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_prev_comp = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_scored_meta_prev_comps_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_xeno_canto = pd.read_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_nodupl_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})

In [4]:
all_birds = list(
    set(chain(*train_metadata["all_labels"])) | 
    set(chain(*train_metadata_prev_comp["all_labels"])) | 
    set(chain(*train_metadata_xeno_canto["all_labels"]))
)

In [5]:
len(all_birds)

1194

In [7]:
bird2int = {
    bird:idx for idx, bird in enumerate(all_birds)
}

In [8]:
write_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_scoed_add_data.json", bird2int)